# collaboration with Woosok Moon
- why would the climate show 1/f noise characteristics?
    - self-organized criticality can lead to 1/f noise
    - several AR(1) processes can add to a locally 1/f spectrum
- What is the connection between the tropical Pacific and the Southern Ocean?
    - heat function
    

## data used by Woosok and colleagues
> OBS Data : NOAA ERSSTv5 (1958-2018), which consists of SODA2.1.6 (0.5 * 0.5) spanning from 1958 to 2008 and GODAS (1.0 * 0.333) from 2009 to 2018.
> Model Data : Large Ensemble Community Project (1800 year simulation with full-coupled models, 1*1 degree.)

## Overview

1. creating PDO and SOi time series
2. spectra of time series
3. "observed" SST anomalies in 3 periods
4. SOi time series from observsations
5. started with regression plots

## To Do
- zonal wind (stress) analysis (South polar plot)
- surface pressure (South polar plot)
- meridional surface velocity ((sub-) tropical Eastern Pacific + (sub-) tropical Atlantic)
- ocean heat content zonally/ vertically integrated

- ask Park about detrending

In [ ]:
import os
import sys
import numpy as np
import xesmf as xe
import cftime
import xarray as xr
import pandas as pd
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
matplotlib.rc_file('../rc_file')
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from tqdm import tqdm_notebook
from maps import rect_polygon
from paths import path_results, path_prace, path_data
from filters import lowpass
from regions import SST_index_bounds, mask_box_in_region
from xr_regression import xr_lintrend, xr_linear_trends_2D, datetime_to_float
from xr_DataArrays import xr_AREA
from ab_derivation_SST import DeriveSST as DS
from bb_analysis_timeseries import AnalyzeTimeSeries as ATS
from ba_analysis_dataarrays import AnalyzeDataArray as ADA

# PDO vs SO index time series in observations, high, and low resolution CESM

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=275))
ax.stock_img()
for i, bounds in enumerate([SST_index_bounds('PDO'), (-30,80,-80,-50)]):
    ax.add_patch(mpl.patches.Polygon(xy=rect_polygon(bounds),
                                  facecolor='none', edgecolor=f'C{i}',
                                  linewidth=2, zorder=2,
                                  transform=ccrs.PlateCarree(), ), )
plt.title('PDO and SOi regions')
plt.tight_layout()

### PDO time series

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y%m')
PDO = pd.read_csv(f'{path_data}/PDO/PDO_NCDC_NOAA.csv', skiprows=1, index_col=0, date_parser=dateparse).to_xarray().to_array().rename({'Date':'time'}).squeeze()
PDO['time'] = PDO.time.dt.year.astype(np.float64)+(PDO.time.dt.month/12-1/24).astype(np.float64)

In [ ]:
PC1_had  = -xr.open_dataset(f'{path_prace}/SST/PMV_EOF_20N_had.nc', decode_times=False).pcs.sel(mode=0)
PC1_ctrl = xr.open_dataset(f'{path_prace}/SST/PMV_EOF_20N_ctrl_51_301.nc', decode_times=False).pcs
PC1_lpd  = xr.open_dataset(f'{path_prace}/SST/PMV_EOF_20N_lpd_154_404.nc', decode_times=False).pcs.sel(mode=0)

### SO index time series
30W-80E, 50-80S

In [ ]:
fn = f'{path_results}/Woosok/PDO_SOi_timeseries.nc'
if os.path.exists(fn):
    ds = xr.open_dataset(fn)
else:
    SST_ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_ctrl_51_301.nc', decode_times=False)
    SST_ctrl = DS().shift_ocn_rect(SST_ctrl)
    SST_lpd  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_lpd_154_404.nc', decode_times=False)
    SST_had  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_had.nc'        , decode_times=False)

    AREA_rect = xr_AREA(domain='ocn_rect')
    AREA_low  = xr_AREA(domain='ocn_low')
    AREA_had  = xr_AREA(domain='ocn_had')

    SOi_MASK_had  = mask_box_in_region(domain='ocn_had' , mask_nr=1, bounding_lats=(-80,-50), bounding_lons=(0,80))
    SOi_MASK_rect = mask_box_in_region(domain='ocn_rect', mask_nr=1, bounding_lats=(-80,-50), bounding_lons=(0,80))
    SOi_MASK_low  = mask_box_in_region(domain='ocn_low' , mask_nr=1, bounding_lats=(-80,-50), bounding_lons=(0,80))

    # had
    AREA_had_SO = AREA_had.where(SOi_MASK_had).sum(dim=['latitude','longitude'])
    print(AREA_had_SO.values)
    SOi_had = (AREA_had*SST_had).where(SOi_MASK_had).sum(dim=['latitude','longitude'])/AREA_had_SO
    SOi_had /= SOi_had.std(dim='time')

    # lpd
    AREA_low_SO = AREA_low.where(SOi_MASK_low).sum(dim=['nlat','nlon'])
    print(AREA_low_SO.values)
    SOi_lpd = (AREA_low*SST_lpd).where(SOi_MASK_low).sum(dim=['nlat','nlon'])/AREA_low_SO
    SOi_lpd /= SOi_lpd.std(dim='time')

    # ctrl
    AREA_rect_SO = AREA_rect.where(SOi_MASK_rect).sum(dim=['t_lat','t_lon'])
    print(AREA_rect_SO.values)
    SOi_ctrl = (AREA_rect*SST_ctrl).where(SOi_MASK_rect).sum(dim=['t_lat','t_lon'])/AREA_rect_SO
    SOi_ctrl /= SOi_ctrl.std(dim='time')

    ds = xr.Dataset()
    ds['CESM_high_PDO'] = PC1_ctrl.squeeze().assign_coords(time=np.arange(3000))
    ds['CESM_high_SOi'] = SOi_ctrl.squeeze().assign_coords(time=np.arange(3000))
    ds['CESM_low_PDO']  = PC1_lpd .squeeze().assign_coords(time=np.arange(3000))
    ds['CESM_low_SOi']  = SOi_lpd .squeeze().assign_coords(time=np.arange(3000))
    ds['HadISST_PDO']   = PC1_had .assign_coords(time=np.arange(1788))
    ds['HadISST_SOi']   = SOi_had .assign_coords(time=np.arange(1788))
    ds.to_netcdf(fn)

In [ ]:
f, ax = plt.subplots(3,1, figsize=(10,7))
for i, run in enumerate(['had', 'ctrl', 'lpd']):
    ax[i].axhline(0, lw=.5, c='k')
    key = ['HadISST', 'CESM_high', 'CESM_low'][i]
    offset = [1870,51,154][i]
    tss = [ds[f'{key}_PDO'].dropna(dim='time'), ds[f'{key}_SOi'].dropna(dim='time')]
    cor_f = np.corrcoef(lowpass(tss[0], 13*12)[7*12:-7*12].values.flatten(), lowpass(tss[1], 13*12)[7*12:-7*12].values.flatten())[0,1]
    cor   = np.corrcoef(tss[0].values.flatten(), tss[1].values.flatten())[0,1]
    for j, ts in enumerate(tss):
        ax[i].plot(ts.time/12+offset, ts, lw=.5, alpha=.5, c=f'C{j}')
        ax[i].plot(ts.time[7*12:-7*12]/12+offset, lowpass(ts, 13*12)[7*12:-7*12], c=f'C{j}', label=['PDO', 'SOi'][j])
    ax[i].plot([0,1],[0,1], c='darkgrey', lw=.5, alpha=.5, label=f'unfiltered corr={cor:.2f}')
    ax[i].plot([0,1],[0,1], c='darkgrey', label=f'13 year lowpass corr={cor_f:.2f}')
    ax[i].legend(loc=2, ncol=2)
    ax[i].set_xlim([(1845,2105),(45,305),(150,410)][i])
    ax[i].set_ylim((-3.4,3.4))
    
ax[0].set_ylabel('two-factor detrended HadISST')
ax[0].set_xlabel('time [years C.E.]')
ax[1].set_ylabel('high res. CESM')
ax[2].set_ylabel('low res. CESM')
for i in np.arange(1,3):  ax[i].set_xlabel('time [model years]')
plt.tight_layout()
plt.savefig(f'{path_results}/Woosok/PDO_SOi_overview')

## MF-TWDFA
left: "observational PDO indices (blue) and a CESM model-constructed one (red)";
right: spectra of time series from high and low resolution simulations
<p float="left">
    <img src="../../results/Woosok/Moon_spectrum.png" width="300"/>
    <img src="../../results/Woosok/CESM_spectra.png" width="300"/>
</p>

### spectral density estimation

In [ ]:
f, ax = plt.subplots(2,2, figsize=(8,5), constrained_layout=True, sharey='row', sharex=True)
for i, run in enumerate(['had', 'ctrl', 'lpd']):
#     ax[].axhline(0, lw=.5, c='k')
    key = ['HadISST', 'CESM_high', 'CESM_low'][i]
    for j, idx in enumerate(['PDO', 'SOi']):
        ts = ds[f'{key}_{idx}'].dropna(dim='time')
        freq, Pxx = ATS(ts).periodogram()
        ax[0,j].loglog(freq, Pxx*10**i, label=f'{idx} {key}', alpha=.6)
        freq, Pxx = ATS(ts).mtspectrum()
        ax[1,j].loglog(freq, Pxx*10**i, label=f'{idx} {key}')
ax[0,0].set_ylim((1e-4,1e5))
for i in range(2):
    ax[i,0].set_ylabel(['periodogram', 'multi-taper spectra'][i])
    ax[0,i].set_title(['PDO', 'SOi'][i])
    ax[1,i].set_xlabel(r'frequency [month$^{-1}$]')
    for j in range(2):
        ax[i,j].legend()

Should 1/f noise not manifest itself in a periodogram/spectral density estimate as a negatively sloped straight line?
Then why do the spectra resemble red noise with a flat and a sloped part?

## SST anomalies in the three periods defined by Woosok

![title](../../results/Woosok/Woosok_SST_lat_lon.png)

In [ ]:
SST_had_orig = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_had.nc')
SST_ersst = xr.open_dataset(f'{path_data}/ersst/ersst.v5.1854-2007.nc').sst#, decode_times=False).sst.assign_coords(time=pd.date_range(start='1854-01-01', end='2010-02-27', freq='M'))
SST_had = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_had.nc')

### HadISST minus mean SST 1958-2018

In [ ]:
SST_had_orig_anom = SST_had_orig.sel(time=slice(f'1958-01-01',f'2018-12-31')) - SST_had_orig.sel(time=slice(f'1958-01-01',f'2018-12-31')).mean(dim='time')
SST_ersst_anom    = SST_ersst.sel(time=slice(cftime.Datetime360Day(year=1958, month=1, day=1),cftime.Datetime360Day(year=2018, month=12, day=30))) -\
                    SST_ersst.sel(time=slice(cftime.Datetime360Day(year=1958, month=1, day=1),cftime.Datetime360Day(year=2018, month=12, day=30))).mean(dim='time')

In [ ]:
%%time
# hadisst: 2min 36s, ersst: 32s
for i, dataset in tqdm_notebook(enumerate(['HadISST', 'ERSST'])):
    fn = f'{path_prace}/SST/{dataset}_lin_detr_58-18.nc'
    if os.path.exists(fn):
        print(f'{fn} exists')
        if i==0:
            SST_lin_detr_had   = xr.open_dataarray(fn)
            SST_lin_detr_had.time.values = SST_had_orig_anom.time.values
        if i==1:
            SST_lin_detr_ersst = xr.open_dataarray(fn)
            SST_lin_detr_ersst.time.values = SST_ersst_anom.time.values
    else:
        da = [SST_had_orig, SST_ersst][i]
        da = da.sel(time=slice(f'1958-01-01',f'2018-12-30'))
        dim_names = [('latitude', 'longitude'), ('lat', 'lon')][i]
        lin_trend = xr_linear_trends_2D(da, dim_names=dim_names, with_nans=True)
        da, time_ = datetime_to_float(da)
        SST_detr = da.copy(data=da.values - (da.time*lin_trend.squeeze()).values)
        SST_detr -= SST_detr.mean(dim='time')
        SST_detr.time.values = time_        
        SST_detr.to_netcdf(fn)

In [ ]:
for i, SST in enumerate([SST_had_orig_anom, SST_ersst_anom, SST_lin_detr_had , SST_lin_detr_ersst, SST_had]):  #
    f = plt.figure(figsize=(12,4.2))
    if i in [0,2,4]:  lats, lons = SST.latitude, SST.longitude
    if i in [1,3]:    lats, lons = SST.lat, SST.lon
    lons, lats = np.meshgrid(lons, lats)
    for j, period in enumerate([(1958,1976),(1977,2000),(2001,2018)]):
        if i in [0,2,4]:  SST_anomaly = SST.sel(time=slice(f'{period[0]}-01-01',f'{period[1]}-12-31')).mean(dim='time')
        if i in [1,3]:    SST_anomaly = SST.sel(time=slice(cftime.Datetime360Day(year=period[0], month=1, day=1),cftime.Datetime360Day(year=period[1], month=12, day=30))).mean(dim='time')
        ax = f.add_subplot(1, 4, j+1, projection=ccrs.Orthographic(central_latitude=-90, central_longitude=180))
        ax.text(.5, 1.05, f'{period[0]}-{period[1]}', transform=ax.transAxes, fontsize=16, ha='center')
        ax.set_position([.03+j*.305,.01,.30,.9])
        im = ax.pcolormesh(lons, lats, SST_anomaly, cmap='cmo.balance',
                           vmin=-.4, vmax=.4, transform=ccrs.PlateCarree())
        ax.coastlines(resolution='110m')
        ax.gridlines()
        ax.add_patch(mpl.patches.Polygon(xy=rect_polygon((-30,80,-80,-50)),
                                      facecolor='none', edgecolor=f'C1',
                                      linewidth=2, zorder=2,
                                      transform=ccrs.PlateCarree(), ), )
    ax = f.add_subplot(1, 4, 4)
    ax.set_position([.947,.1,.017,.75])
    cbar = plt.colorbar(im, cax=ax, shrink=.9, pad=.0, orientation='vertical', extend='both')
    f.text(0.007,0.5, ['HadISST - mean(\'58-\'18)', 'ERSST - mean(\'58-\'18)',\
                       'HadISST - lintrend(\'58-\'18)', 'ERSST - lintrend(\'58-\'18?)',\
                       'HadISST two-factor detrended'][i], rotation=90, fontsize=16,va='center')
# cbar.ax.set_yticklabels(np.arange(-.4,.5,.1), fontsize=12)

- need to update boundary
- anomaly pattern depends on detrending
- third row most like Woosok's image (also same dataset)

### SOi time series

In [ ]:
# 
latS, latN, lonW, lonE = -80, -50, -30, 80
SOi_had = SST_had.sel(longitude=slice(lonW,lonE)).sel(latitude=slice(latN,latS)).mean(dim=('latitude', 'longitude'))
SOi_had_orig = SST_had_orig.sel(longitude=slice(lonW,lonE)).sel(latitude=slice(latN,latS)).mean(dim=('latitude', 'longitude'))
SOi_had_orig_anomaly = SOi_had_orig - SOi_had_orig.sel(time=slice(pd.datetime(year=1958, month=1, day=1),pd.datetime(year=2018, month=12, day=31))).mean(dim='time')
SOi_ersst = SST_ersst.sel(lon=slice(lonE,360+lonW)).sel(lat=slice(latS, latN)).mean(dim=('lat', 'lon'))
SOi_ersst_anomaly = SOi_ersst - SOi_ersst.sel(time=slice(cftime.Datetime360Day(year=1954, month=1, day=1),cftime.Datetime360Day(year=2011, month=2, day=27))).mean(dim='time')

In [ ]:
f, ax = plt.subplots(3, 1, figsize=(8,5), constrained_layout=True)
for i in range(3):  ax[i].axhline(0, lw=.5, c='k')
ax[1].set_title('')
ax[0].set_ylabel('ERSST')
ax[0].plot(SOi_ersst_anomaly.time, SOi_ersst_anomaly, alpha=0.3)
ax[0].plot(SOi_ersst_anomaly.time[5*12:-5*12], lowpass(SOi_ersst_anomaly,12*5)[5*12:-5*12])
ax[0].set_xlim((cftime.Datetime360Day(year=1850, month=1, day=1), cftime.Datetime360Day(year=2020, month=1, day=1)))

ax[1].set_ylabel('HadISST')
ax[1].plot(SOi_had_orig_anomaly.time, SOi_had_orig_anomaly, alpha=.3)
ax[1].plot(SOi_had_orig_anomaly.time[5*12:-5*12], lowpass(SOi_had_orig_anomaly,12*5)[5*12:-5*12])
ax[1].set_xlim(pd.datetime(year=1850, month=1, day=1),pd.datetime(year=2020, month=1, day=1))

ax[2].set_ylabel('HadISST\ntwo-factor detrended')
ax[2].plot(SOi_had.time, SOi_had, alpha=.3)
ax[2].plot(SOi_had.time[5*12:-5*12], lowpass(SOi_had,12*5)[5*12:-5*12])
ax[2].set_xlim(pd.datetime(year=1850, month=1, day=1),pd.datetime(year=2020, month=1, day=1))

# plt.gca().axvspan(pd.datetime(year=1958, month=1, day=1),pd.datetime(year=1976, month=12, day=31), alpha=0.5, color='g')
# plt.gca().axvspan(pd.datetime(year=1977, month=1, day=1),pd.datetime(year=2000, month=12, day=31), alpha=0.5, color='red')
# plt.gca().axvspan(pd.datetime(year=2001, month=1, day=1),pd.datetime(year=2018, month=12, day=31), alpha=0.5, color='g')

with 5 year lowpass filter

- It is not clear to me is the sudden warming in 1940 is real.

#### detrending only from 1958 onwards

In [ ]:
PDO = pd.read_csv(f'{path_data}/PDO/PDO_NCDC_NOAA.csv', skiprows=1, index_col=0, date_parser=dateparse).to_xarray().to_array().rename({'Date':'time'}).squeeze()

In [ ]:
f, ax = plt.subplots(1,2, figsize=(8,3), constrained_layout=True, sharey=True)
for i, SOi_anomaly in enumerate([SOi_had_orig_anomaly, SOi_ersst_anomaly]):
    last60y = SOi_anomaly.isel(time=slice(-60*12,-1))
    last60y.plot(ax=ax[i], lw=.5, c=f'C0', alpha=.3)
    lowpass(last60y,12*5)[12*5:-12*5].plot(ax=ax[i], c=f'C0')
    (last60y-xr_lintrend(last60y)).plot(ax=ax[i], c=f'C1', lw=.5, alpha=.3)
    lowpass(last60y-xr_lintrend(last60y), 12*5)[12*5:-12*5].plot(ax=ax[i], c=f'C1')
    ax[i].axhline(0, lw=.5, c='k')
    xr_lintrend(last60y).plot(ax=ax[i])
    ax[i].set_ylabel(['SOi',''][i])

## identifying time periods

![title](../../results/Woosok/Woosok_phases.png)

SOi (solid), PDO (dashed)

#### SOi index region trende
- there is almost no trend in the HadISST dataset from 1958
- noticable trend in ERSST from 1949
- what is the smoothing used originally?

In [ ]:
f, ax = plt.subplots(2,1, figsize=(8,5), constrained_layout=True)
for i in range(2):
    ax[i].axhline(0, lw=.5, c='k')
    ax[i].axvline(1958, lw=.5, c='grey')
    ax[i].axvline(1980, lw=.5, c='grey')
    ax[i].axvline(2009, lw=.5, c='grey')
key = 'HadISST'
for j, ts in enumerate([-ds[f'{key}_PDO'].dropna(dim='time'), ds[f'{key}_SOi'].dropna(dim='time')]):
    ax[j].plot(ts.time[7*12:-7*12]/12+1870, lowpass(ts, 10*12)[7*12:-7*12], label=['PDO two-factor detrended', 'SOi two-factor detrended'][j])
ax[0].set_xlim((1957.5,2018))
ax[0].plot(PDO.time.dt.year+PDO.time.dt.month/12, lowpass(PDO,10*12), label='PDO from NOAA')
ax[1].plot(SST_had_orig.time.dt.year+SST_had_orig.time.dt.month/12, lowpass(SOi_had_orig_anomaly,10*12)*10, label=r'SOi original ($\times$10)')
ax[1].plot(SST_had_orig.time.dt.year.isel(time=slice(-60*12,-1))+SST_had_orig.time.dt.month.isel(time=slice(-60*12,-1))/12,\
#         SOi_had_orig_anomaly[-60*12:]-xr_lintrend(SOi_had_orig_anomaly[-60*12:])*10,\
        lowpass(SOi_had_orig_anomaly.isel(time=slice(-60*12,-1))-xr_lintrend(SOi_had_orig_anomaly.isel(time=slice(-60*12,-1))),10*12)*10,\
        label=r'SOi original lin detrended ($\times$10)')
last60y = SOi_ersst_anomaly.isel(time=slice(-60*12,-1))
ax[1].plot(last60y.time.dt.year[12*5:-12*5]+last60y.time.dt.month[12*5:-12*5]/12, lowpass(last60y-xr_lintrend(last60y), 12*5)[12*5:-12*5]*5, label=r'ERSST-lintrend ($\times$5)')
for i in range(2):
    ax[i].legend(ncol=2, fontsize=8)
    ax[i].set_ylabel(['PDO','SOi'][i])
    ax[i].set_ylim((-2,2))
    ax[i].set_xlim((1958,2018))
ax[1].set_xlabel('time [years C.E.]')


- it is not clear how the data was detrended in Woosok's case, this is very important for the SOi (cf first crossover point)
- PDO: similat between detrended & original & Woosok's

### lead-lag  analysis between signals
left and middle:  cold and warm observations; 
right: warm CESM 

<p float="left">
    <img src="../../results/Woosok/SST_obs_cold.png" width="300"/>
    <img src="../../results/Woosok/SST_obs_warm.png" width="300"/>
    <img src="../../results/Woosok/SST_CESM_warm.png" width="300"/>
</p>

#### remarks by Woosok:
- model amplitude much smaller than in obs.
- model does not show Atlantic negative signal

my own remarks
- hard to compare with different time lags...
- how is the negative generated? as a composite of points A&C?

In [ ]:
SST_ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_ctrl_51_301.nc', decode_times=False)
SST_ctrl = DS().shift_ocn_rect(SST_ctrl)
SST_lpd  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_lpd_154_404.nc', decode_times=False)

In [ ]:
SOi = lowpass(ds['CESM_high_SOi'], 12*10)

In [ ]:
SOi.time.values = SST_ctrl.time.values

In [ ]:
%%time
# 8 min 57s
lagged_SST = {}
for lagx in [0,5,10,15,20]:
    lagged_SST[f'SOi_SST_lag{lagx}'] = ADA().lag_linregress(x=SOi, y=SST_ctrl, lagx=lagx*12, lagy=0)

In [ ]:
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = matplotlib.path.Path(verts * radius + center)

In [ ]:
lats, lons = SST_ctrl.t_lat, SST_ctrl.t_lon; lons, lats = np.meshgrid(lons, lats)
maxv, ticks = .5, np.arange(-.5,.51,.25)
f = plt.figure(figsize=(4, 10.5), constrained_layout=True)
for i, lagx in enumerate([0,5,10,15,20]):
    for j in range(2):
        ax = f.add_subplot(5, 2, j+i*2+1, projection=ccrs.LambertAzimuthalEqualArea(central_latitude=-90))#, central_longitude=-179.9))
        ax.set_position([.01+j*.5,.05+i*.19,.48,.19])
        ax.set_boundary(circle, transform=ax.transAxes)
        ax.set_extent([-180, 180, -90, -[50,0][j]], ccrs.PlateCarree())
        im = ax.pcolormesh(lons, lats, lagged_SST[f'SOi_SST_lag{lagx}'].cor, cmap='cmo.balance',
                           vmin=-maxv, vmax=maxv, transform=ccrs.PlateCarree())
        if j==0:  ax.text(.01,.95, f'-{lagx} yrs', transform=ax.transAxes)
        ax.coastlines(resolution='110m')
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
        gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,181,60))
        gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(-90,91,30))
        
ax = f.add_axes([.05,.025,.9,.015])
cbar = plt.colorbar(im, cax=ax, shrink=.9, pad=.0, orientation='horizontal', extend='both', ticks=ticks, label='correlation')
cbar.ax.set_yticklabels(ticks, fontsize=12)

- at 0 lag tha patter is quite similar
- at 20 year lag, the observsations do not show positive Amundsen/Bellingshausen Sea correlation
- some hints of wavenumber 3, at 20 years lag wavenumber 1

In [ ]:
t = np.linspace(0,10,100)
X = xr.DataArray(data=np.sin(t), coords={'time':t}, dims='time')
Y = xr.DataArray(data=np.cos(t), coords={'time':t}, dims='time')
X.plot()
Y.plot()
plt.figure()
for lagx in np.arange(-20,20):
    plt.scatter(lagx, ADA().lag_linregress(x=X, y=Y, lagx=lagx, lagy=0).cor.values)

### identifying times in the model

### OHC anomalies

- still needs to be done

### sea level pressure

<img src="../../results/Woosok/P_surface.png" width="600"/>

In [ ]:
# %%time
# from paths import path_prace_prace
# # ca. 12min for single dataset loading and concatenation
# Vs = []
# Us = []
# Ps = []
# for i, (y,m,fn) in enumerate(IterateOutputCESM(domain='atm', run='ctrl', tavg='monthly')):
#     if y>50:
#         if m==1: print(y)
# #         Vs.append(xr.open_dataset(fn, decode_times=False).V[0,-1,:,:])
# #         Us.append(xr.open_dataset(fn, decode_times=False).U[0,-1,:,:])
#         Ps.append(xr.open_dataset(fn, decode_times=False).PS[0,:,:])
# ds = xr.concat(Ps, dim='time', coords='minimal')
# ds.to_netcdf(f'{path_prace}/Woosok/PS_monthly.nc')

In [ ]:
PS = xr.open_dataarray(f'{path_prace}/Woosok/PS_monthly.nc', decode_times=False).isel(time=slice(0,3000))
PS_regr = ADA().lag_linregress(x=SOi, y=PS, lagx=0, lagy=0)

In [ ]:
lats, lons = PS.lat, PS.lon; lons, lats = np.meshgrid(lons, lats)

In [ ]:
f = plt.figure(figsize=(4.3, 2), constrained_layout=True)
f.suptitle(r'P$_{surface}$')
for i in range(2):
    ax = f.add_subplot(1, 3, i+1, projection=ccrs.LambertAzimuthalEqualArea(central_latitude=-90))#, central_longitude=-179.9))
    ax.set_position([.005+i*.47,.01,.47,.98])
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.set_extent([-180, 180, -90, -30], ccrs.PlateCarree())
    if i==0:
        ax.pcolormesh(lons, lats, PS.mean(dim='time'), cmap='cmo.dense',
                       vmin=6e4, vmax=1.1e5, transform=ccrs.PlateCarree())
        ax.text(.05,.95, 'mean', transform=ax.transAxes)
    if i==1: 
        im = ax.pcolormesh(lons, lats, PS_regr.cor, cmap='cmo.balance',
                           vmin=-.1, vmax=.1, transform=ccrs.PlateCarree())
        ax.text(.15,.95, 'corr.', transform=ax.transAxes)

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,181,60))
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(-90,91,30))

ax = f.add_axes([.96,.05,.02,.9])
cbar = plt.colorbar(im, cax=ax, shrink=.9, pad=.0, orientation='vertical', extend='both', ticks=ticks, label='correlation')
cbar.ax.set_yticklabels(ticks, fontsize=12)

- compare to middle panel
- different pattern around Antarctic coast negative in obs., positive in CESM
- similar negative pattern in mid-latitude Pacific and South of Australia

### surface winds
left: zonal wind, right: meridional wind
<p float="center">
    <img src="../../results/Woosok/Vwind.png" width="450"/>
    <img src="../../results/Woosok/Uwind.png" width="450"/>
</p>
<named exactly wrong here>

In [ ]:
U = xr.open_dataarray(f'{path_prace}/Woosok/U_monthly.nc', decode_times=False).isel(time=slice(0,3000))
V = xr.open_dataarray(f'{path_prace}/Woosok/V_monthly.nc', decode_times=False).isel(time=slice(0,3000))

In [ ]:
%%time
# 48.5 s
U_regr = ADA().lag_linregress(x=SOi, y=U, lagx=0, lagy=0)
V_regr = ADA().lag_linregress(x=SOi, y=V, lagx=0, lagy=0)

In [ ]:

for k, regr in enumerate([U_regr, V_regr]):
    for j, quant in enumerate(['cor']):#, 'cov']):
        f = plt.figure(figsize=(4.3, 2), constrained_layout=True)
        f.suptitle(['U ', 'V '][k])
        for i in range(2):
            ax = f.add_subplot(1, 3, i+1, projection=ccrs.LambertAzimuthalEqualArea(central_latitude=-90))#, central_longitude=-179.9))
            ax.set_position([.005+i*.47,.01,.47,.98])
            ax.set_boundary(circle, transform=ax.transAxes)
            ax.set_extent([-180, 180, -90, -30], ccrs.PlateCarree())
            if i==0:
                ax.pcolormesh(lons, lats, [U,V][k].mean(dim='time'), cmap='cmo.curl',
                               vmin=-15, vmax=15, transform=ccrs.PlateCarree())
                ax.text(.05,.95, 'mean', transform=ax.transAxes)
            if i==1: 
                im = ax.pcolormesh(lons, lats, regr[quant], cmap='cmo.balance',
                                   vmin=-.1, vmax=.1, transform=ccrs.PlateCarree())
                ax.text(.15,.95, quant, transform=ax.transAxes)

            ax.coastlines(resolution='110m')
            gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
            gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,181,60))
            gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(-90,91,30))


        ax = f.add_axes([.96,.05,.02,.9])
        cbar = plt.colorbar(im, cax=ax, shrink=.9, pad=.0, orientation='vertical', extend='both', ticks=ticks, label='correlation')
        cbar.ax.set_yticklabels(ticks, fontsize=12)

left subplots is mean of winds

right subplots need to be compared to warm phase (middle image)

- zonal wind U: negative correlations in CESM, around Antarctica, but positive in "observations"
- meridional wind V: wavenumber 3 visible in correct locations

### surface meridional Ekman transport 
<img src="../../results/Woosok/VVEL_Pacific.png" width="550"/>
<img src="../../results/Woosok/VVEL_Atlantic.png" width="550"/>

- wavenumber 3 pattern in  zonal wind (found in _Wang & Dommenget (2015)_)

## References:
_Wang & Dommenget (2015)_: The leading modes of decadal SST variability in the Southern Ocean in CMIP5 simulations